Purpose: Prepare data for XGBoost regression.<br>
Author: Anna Pardo<br>
Date initiated: Dec. 19, 2025

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from functools import reduce

In [2]:
# load Yg TPM
mdtpm = pd.read_csv("~/Yucca_genomics/rna_insilico_genome/TPM/Yg_toYgIS_allTPM_correctedmd_over1mil.txt",sep="\t",header="infer")
mdtpm.head()

/tmp/ipykernel_256/2025937544.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  mdtpm = pd.read_csv("~/Yucca_genomics/rna_insilico_genome/TPM/Yg_toYgIS_allTPM_correctedmd_over1mil.txt",sep="\t",header="infer")


,sample_name,genotype,time,treat,ZT,species,Yucal.01G000100.v2.1,Yucal.01G000200.v2.1,Yucal.01G000300.v2.1,Yucal.01G000400.v2.1,...,YufilH1095122m.g,YufilH1095123m.g,YufilH1095125m.g,YufilH1095126m.g,YufilH1095128m.g,YufilH1095131m.g,YufilH1095132m.g,YufilH1095134m.g,YufilH1095146m.g,YufilH1095147m.g
0,Y1,18,1.0,W,1.0,gloriosa,34.002815,4.546167,0.0,17.236119,...,0.000000,6.167441,1.435253,0.279077,11.420139,0.662252,1.886709,6.675105,0.0,0.000000
1,Y10,2AB,1.5,W,3.0,gloriosa,40.070758,3.628454,0.0,14.918115,...,0.713535,2.197522,15.695904,1.193256,10.172780,0.000000,2.214483,12.125756,0.0,0.000000
2,Y100,2AB,6.5,W,23.0,gloriosa,47.402599,5.201760,0.0,17.406497,...,0.314746,2.261806,21.742515,1.798380,10.256681,1.748663,2.075757,25.215640,0.0,1.838780
3,Y101,2AB,1.0,D,1.0,gloriosa,57.062380,6.374324,0.0,10.567561,...,0.000000,1.072366,27.573939,1.164591,9.105769,1.257431,2.431447,4.508369,0.0,0.813682
4,Y103,2AB,1.0,D,1.0,gloriosa,34.679279,6.087451,0.0,11.115252,...,0.000000,0.914550,28.621412,0.869052,7.076224,0.515567,2.419222,4.625880,0.0,0.867418


In [3]:
mdtpm.drop(["time","species"],axis=1,inplace=True)

In [4]:
mdtpm["genotype"] = mdtpm["genotype"].astype(str)
mdtpm["genotype"].unique()

array(['18', '2AB', '1AB', '19', '15', 'Eudy', 'G', '56', '36', '13',
       '45', '52', '43', '37', '48', '55', '70', '61', '51', '46', '53',
       '16', '6', '12', '20', '50'], dtype=object)

In [5]:
zttpm = mdtpm[mdtpm["ZT"].isin([1.0,5.0,9.0,13.0,17.0,21.0])]

In [6]:
mdonly = zttpm[["sample_name","genotype","treat","ZT"]]

In [7]:
# check how many and which genotypes have complete data (at least 1 replicate in each treatment+ZT)
def iscomplete(gt):
    gtmd = mdonly[mdonly["genotype"]==gt]
    gtcount = gtmd.groupby(["treat","ZT"]).count().reset_index()[["treat","ZT","sample_name"]].rename(columns={"sample_name":"nreps"})
    if len(gtcount.index)==12:
        return "complete"
    elif len(gtcount.index)<12:
        return "incomplete"

In [8]:
dfdict={"Genotype":[],"Complete":[]}
for g in mdonly["genotype"].unique():
    dfdict['Genotype'].append(g)
    dfdict['Complete'].append(iscomplete(g))

In [9]:
cdf = pd.DataFrame(dfdict)

In [10]:
cdf[cdf["Complete"]=="incomplete"]

,Genotype,Complete
4,15,incomplete
11,52,incomplete
15,55,incomplete
16,70,incomplete
20,53,incomplete
21,16,incomplete
24,20,incomplete
25,50,incomplete


In [11]:
def iscompletedf(gt):
    gtmd = mdonly[mdonly["genotype"]==gt]
    gtcount = gtmd.groupby(["treat","ZT"]).count().reset_index()[["treat","ZT","sample_name"]].rename(columns={"sample_name":"nreps"})
    return gtcount

In [12]:
complete_gts = list(cdf[cdf["Complete"]=="complete"]["Genotype"])

In [13]:
len(complete_gts)

18

In [14]:
comptpm = zttpm[zttpm["genotype"].isin(complete_gts)]

In [15]:
list(cdf[cdf["Complete"]=="incomplete"]["Genotype"])

['15', '52', '55', '70', '53', '16', '20', '50']

In [16]:
# for comptpm: filter out genes that are expressed in <10% of genotypes (i.e., expressed only in 1 or 2 genotypes)
def egtnum(gid):
    tpmsub = comptpm[["genotype",gid]]
    maxtpm = tpmsub.groupby("genotype").max().reset_index()
    # count the number of genotypes with max>0
    max0 = maxtpm[maxtpm[gid]>0]
    return len(max0)

In [17]:
dropgenes = []
for c in comptpm.columns:
    if c.startswith("Y"):
        if egtnum(c)<=2:
            dropgenes.append(c)

In [18]:
len(dropgenes)

4143

In [19]:
len(comptpm.columns)-4

85962

In [20]:
filtcomp = comptpm.drop(dropgenes,axis=1)

In [21]:
len(filtcomp.columns)-4

81819

In [22]:
# drop zero-variance genes
def variance_threshold_selector(data):
    selector = VarianceThreshold()
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [23]:
filttpm = filtcomp.set_index(["sample_name","genotype","treat","ZT"])

In [24]:
vttpm = variance_threshold_selector(filttpm)

In [25]:
print("Number of zero-variance genes removed:",len(filttpm.columns)-len(vttpm.columns))

Number of zero-variance genes removed: 0


In [26]:
vttpm_log = vttpm.apply(lambda x: np.log2(x+1))

In [27]:
vttpm_log.reset_index(inplace=True)

In [28]:
vttpm_log.drop("sample_name",axis=1,inplace=True)

In [29]:
vttpm_log["treat_ZT"] = vttpm_log["treat"]+"_"+vttpm_log["ZT"].astype(int).astype(str)
vttpm_log.head()

,genotype,treat,ZT,Yucal.01G000100.v2.1,Yucal.01G000200.v2.1,Yucal.01G000300.v2.1,Yucal.01G000400.v2.1,Yucal.01G000500.v2.1,Yucal.01G000600.v2.1,Yucal.01G000700.v2.1,...,YufilH1095122m.g,YufilH1095123m.g,YufilH1095125m.g,YufilH1095126m.g,YufilH1095128m.g,YufilH1095131m.g,YufilH1095132m.g,YufilH1095134m.g,YufilH1095147m.g,treat_ZT
0,18,W,1.0,5.129399,2.471491,0.0,4.188727,2.140948,3.181674,3.168583,...,0.0,2.841458,1.284072,0.355103,3.634609,0.733139,1.529426,2.940186,0.000000,W_1
1,2AB,D,1.0,5.859532,2.882511,0.0,3.532013,2.045877,3.441016,2.950223,...,0.0,1.051279,4.836628,1.114095,3.337107,1.174682,1.778817,2.461625,0.858921,D_1
2,2AB,D,1.0,5.157015,2.825267,0.0,3.598753,2.070301,3.257171,2.777274,...,0.0,0.937005,4.888569,0.902307,3.013681,0.599857,1.773668,2.492079,0.901045,D_1
3,1AB,W,21.0,5.791442,2.753920,0.0,4.225544,1.127059,2.864516,2.795164,...,0.0,1.217595,4.454795,1.659643,3.064140,0.000000,1.809274,4.059519,0.000000,W_21
4,18,W,21.0,5.596679,3.241637,0.0,4.047620,1.632024,3.407077,3.284910,...,0.0,3.086022,1.656996,2.910847,3.955409,1.919294,1.736134,4.441396,0.000000,W_21


In [30]:
vttpm_log.drop(["treat","ZT"],axis=1,inplace=True)

In [31]:
# calculate mean expression of each gene on a per-treatment/ZT basis
## define a function to do this
def genemeans(gid):
    sublog = vttpm_log[["genotype","treat_ZT",gid]]
    sublog.groupby(["genotype","treat_ZT"]).mean().reset_index()
    slp = sublog.pivot_table(index="genotype",columns="treat_ZT",values=gid).reset_index()
    for c in slp.columns:
        if c!="genotype":
            slp.rename(columns={c:gid+"_"+c},inplace=True)
    return slp

In [32]:
dflist = []
for c in vttpm_log.columns:
    if c.startswith("Y"):
        dflist.append(genemeans(c))

In [68]:
dflist[0].head()

treat_ZT,genotype,Yucal.01G000100.v2.1_D_1,Yucal.01G000100.v2.1_D_13,Yucal.01G000100.v2.1_D_17,Yucal.01G000100.v2.1_D_21,Yucal.01G000100.v2.1_D_5,Yucal.01G000100.v2.1_D_9,Yucal.01G000100.v2.1_W_1,Yucal.01G000100.v2.1_W_13,Yucal.01G000100.v2.1_W_17,Yucal.01G000100.v2.1_W_21,Yucal.01G000100.v2.1_W_5,Yucal.01G000100.v2.1_W_9
0,12,4.671265,4.921500,5.005791,4.798222,4.095183,4.459452,4.871347,4.712149,4.941031,5.006829,4.102163,4.456983
1,13,5.144395,4.975804,4.853688,5.012945,4.893604,5.055911,5.258964,5.489978,5.268651,5.200845,4.848472,5.053348
2,18,5.684827,5.435823,5.405006,5.475807,5.751970,5.898819,5.573681,5.574150,5.244135,5.290898,6.075920,5.711747
3,19,5.135665,5.190652,5.463501,5.250987,4.594409,5.076206,5.157064,5.322314,5.205319,4.971768,4.843764,5.160359
4,1AB,5.494941,5.422841,5.219464,4.980347,5.043138,5.326056,5.546432,5.651210,5.358873,5.537470,5.627045,5.456193


In [69]:
len(dflist)

81819

In [33]:
allfeatures = reduce(lambda x,y: pd.merge(x,y,on="genotype"),dflist)

KeyboardInterrupt: 